In [1]:
import anvil.server

anvil.server.connect("PLPREEFX77QBTZXIMMGXT5S5-LLZOQE4RLL7KOL6G")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


In [2]:
# Importing all the utilities
from utilities_stats import *

#Setting the total number of matchdays in the season
tot_giornate = 35

# setting useful parameters
Teams, Logos, parameters, Results_0, goal_marks = set_par()
giornate = current_matchday()

# filling individual dataframes and conditional display
Results = fill_dataframe_partita(Results_0, giornate, parameters, goal_marks, Teams, Print = False)

# saving cumulative statistical data
pf_med, pf_std, ps_med, ps_std, gf_med, gf_std = cumulative_data(Results, giornate, Print = False)

# Building cumulative dataframe
Total = make_Total_df(Results, giornate, goal_marks)
Tot_per_round = partial_totals(Results, giornate, tot_giornate, goal_marks)
#Total.head(10)

In [3]:
import json
from pymongo import MongoClient
from pprint import pprint
import pymongo



with open('credential.json','r') as f:
    cred = json.load(f)
    
    
cluster = MongoClient(cred['cred'])
# choosing database
db = cluster["Game"]
# choosing collection
collection = db["Players"]
collection_man = db['Managers']
collection_tr = db['Transfers']

In [4]:
@anvil.server.callable

def test_funct(owner, squad):
    owner = owner.lower()
    squad = squad.lower()
    players = []
    roles = []
    loan = []
    posts = collection.find({'info.current_team.owner': owner,'info.current_team.squad': squad})
    for player in posts:
        players.append(player['name'])
        roles.append(player['info']['personal_info']['FC_role'])
        temp = ''
        if player['info']['current_team']['on_loan']:
            temp = 'on loan'
        loan.append(temp)
    return players, roles, loan

In [6]:
@anvil.server.callable

def man_data(owner):
    owner = owner.lower()
    dic = collection_man.find_one({'owner': owner})
    dic['main_value'] = 0
    dic['prim_value'] = 0
    for pl in collection.find({'info.current_team.owner': owner,'info.current_team.squad': 'main'}):
        dic['main_value'] += pl['info']['stats']['Qt_A']
    for pl in collection.find({'info.current_team.owner': owner,'info.current_team.squad': 'primavera'}):
        dic['prim_value'] += pl['info']['stats']['Qt_A']
    return dic

In [7]:
@anvil.server.callable
def pl_names():
    posts = collection.find({})
    players = []
    for post in posts:
        players.append(post['name'])
    return sorted(players)

In [8]:
@anvil.server.callable
def pl_info(name):
    dic = collection.find_one({'name': name})
    dic_stats = dic['info']['stats']
    return dic, dic_stats

In [9]:
@anvil.server.callable
def find_trans(name = False, operation = False, year = False, owner = False):
    filt = {}
    if name:
        filt['name'] ={'$regex':''+name,'$options':'i'}
    if operation:
        filt['operation'] = operation
    if year:
        filt['date'] ={'$regex':'^'+str(year),'$options':'i'}
    posts = []
    for dic in collection_tr.find(filt):
        owner = 'N/A'
        if 'owner' in dic.keys():
            owner = dic['owner']
        elif 'new_owner' in dic.keys():
            owner = dic['new_owner']
        elif 'exch_owner' in dic.keys():
            owner = dic['exch_owner']
            
        posts.append({'name': dic['name'],'operation':dic['operation'],'date':dic['date'], 'owner': owner})
    
    return posts

In [16]:
#fortuna_evo(Results, Teams, Tot_per_round)
#C_factor_logos(Total, giornate, Teams, tot_giornate, Logos)
#for premio in ['Caduti','Porta Violata','Catenaccio','Panchina Oro','Cartellino Facile']:
    #premio_plot(Results, giornate, Teams, Logos, premio)